In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
#from plotly.offline import init_notebook_mode, iplot
#import plotly.graph_objs as go
import os
import warnings

# 파일 불러와서 열이름 바꾸기

In [2]:
##### 파일 불러오기 #####
df = pd.read_csv("C:/Users/user/Desktop/2021 빅콘테스트_데이터분석분야_퓨처스리그_ECO제주\data_processing/2_3_short_term_frgn_3_resident_pop_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'visit_pop_cnt', 'total_pop']  ##### 각자 데이터에 맞게 열이름 변경하기 #####
df

,base_date,emd_cd,visit_pop_cnt,total_pop
base_date,,,,
2018-01-31,2018-01-31,50110250,58090.5960,24419
2018-02-28,2018-02-28,50110250,69710.8394,24427
2018-03-31,2018-03-31,50110250,98026.6381,24532
2018-04-30,2018-04-30,50110250,103289.7891,24606
2018-05-31,2018-05-31,50110250,115255.9750,24715
...,...,...,...,...
2021-02-28,2021-02-28,50130620,280.6159,4173
2021-03-31,2021-03-31,50130620,1475.8404,4178
2021-04-30,2021-04-30,50130620,1283.0997,4161


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date', 'visit_pop_cnt'], 1)     ##### 각자 필요없는 열 제거 #####
data

,emd_cd,total_pop
base_date,,
2018-01-31,50110250,24419
2018-02-28,50110250,24427
2018-03-31,50110250,24532
2018-04-30,50110250,24606
2018-05-31,50110250,24715
...,...,...
2021-02-28,50130620,4173
2021-03-31,50130620,4178
2021-04-30,50130620,4161


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110310  ##### 각 행정동코드 입력 #####

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,total_pop
base_date,
2018-01-31,9269
2018-02-28,9276
2018-03-31,9307
2018-04-30,9337
2018-05-31,9406
2018-06-30,9437
2018-07-31,9439
2018-08-31,9416
2018-09-30,9416


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'total_pop': base_date
 2018-01-31    9269
 2018-02-28    9276
 2018-03-31    9307
 2018-04-30    9337
 2018-05-31    9406
 2018-06-30    9437
 2018-07-31    9439
 2018-08-31    9416
 2018-09-30    9416
 2018-10-31    9437
 2018-11-30    9428
 2018-12-31    9447
 2019-01-31    9446
 2019-02-28    9455
 2019-03-31    9484
 2019-04-30    9501
 2019-05-31    9461
 2019-06-30    9476
 2019-07-31    9478
 2019-08-31    9458
 2019-09-30    9423
 2019-10-31    9432
 2019-11-30    9427
 2019-12-31    9405
 2020-01-31    9414
 2020-02-29    9416
 2020-03-31    9409
 2020-04-30    9377
 2020-05-31    9403
 2020-06-30    9467
 2020-07-31    9487
 2020-08-31    9488
 2020-09-30    9457
 2020-10-31    9453
 2020-11-30    9414
 2020-12-31    9422
 2021-01-31    9430
 2021-02-28    9463
 2021-03-31    9436
 2021-04-30    9465
 2021-05-31    9486
 2021-06-30    9495
 Name: total_pop, dtype: int64}

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)
#sub_df_float.tail()

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))
    
##### 여기서 p-value 확인 -> 0.05보다 크면 아래로 내려가서 차분하기 / 작으면 차분 생략(아래 셀 생략) #####

ADF Statistic : -3.764781
p-value : 0.003289
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [8]:
#!pip install pmdarima

In [9]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

##### 여기서 아래쪽에 'Best model:  ARIMA(0,1,0)(0,0,0)[1] intercept' 이렇게 나오면 (0,1,0) 가져다 쓸거임 #####
##### ARIMA(0,1,0) 여기 숫자 세 개만 잘 보면 됨 !!!! #####

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=455.343, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=425.970, Time=0.05 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=403.522, Time=0.12 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=402.909, Time=0.09 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=401.265, Time=0.20 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=398.087, Time=0.19 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=407.600, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=401.794, Time=0.13 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=399.108, Time=0.15 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=403.530, Time=0.14 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=399.538, Time=0.23 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=401.033, Time=0.25 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=407.125, Time=0.29 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=398.316, Time=0.19 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=401.432, Time=0.23 sec
 ARIMA(3,0,0)(0,0,0)[1] i

ARIMA(order=(0, 0, 5), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [10]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.total_pop.values, order=(0,0,5))     ##### 여기 order=(0,1,0)에 위에서 나온 숫자 넣는거 #####
                                    ##### 그리고 sub_df.card_amt.values에서 가운데 card_amt 각자걸로 바꾸기 #####

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(0, 5)   Log Likelihood                -194.043
Method:                       css-mle   S.D. of innovations             21.397
Date:                Sat, 11 Sep 2021   AIC                            402.086
Time:                        01:35:18   BIC                            414.249
Sample:                             0   HQIC                           406.544
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       9423.9940     24.676    381.911      0.000    9375.630    9472.358
ma.L1.y        1.4769      0.084     17.565      0.000       1.312       1.642
ma.L2.y        1.8479        nan        nan        n

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:1521: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(-inv(hess)))


# 예측하기

In [11]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

##### 맨 앞에 나온 'array([10681271.81014634, 10741037.02029268])' 이게 차례대로 7, 8월 예측값임 #####
##### 7월 : 10681271.81014634, 8월 : 10741037.02029268 #####
##### 이 값 가져다 엑셀파일에 입력하기 #####

(array([9474.99245261, 9423.9940233 ]), array([21.3968344 , 38.16413914]), array([[9433.0554278 , 9516.92947743],
       [9349.19368508, 9498.79436153]]))
